In [96]:
import torch
from transformers import AutoTokenizer,BertForMaskedLM 
model = BertForMaskedLM.from_pretrained("hfl/chinese-bert-wwm-ext")#"/remote-home/xtzhang/CTC/CTC2021/SpecialEdition/tmp/sighan_raw/ConfusionCluster/bert/checkpoint-5560")

tokenizer_model_name_path="hfl/chinese-bert-wwm-ext"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name_path )


Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [86]:
#a = tokenizer("或[MASK]是因为人们已渐渐随着时代的迁变而对于人生有了新的想法", return_tensors='pt')
a = tokenizer("或著是因为人们已渐渐随着时代的迁变而对于人生有了新的想法", return_tensors='pt')

In [93]:
a = tokenizer("我今天吃了烤[MASK],真好吃啊", return_tensors='pt')
#a = tokenizer("我今天吃了烤又，真好吃啊", return_tensors='pt')

In [69]:
a

{'input_ids': tensor([[ 101, 2772,  103, 3221, 1728,  711,  782,  812, 2347, 3933, 3933, 7390,
         4708, 3198,  807, 4638, 6810, 1359, 5445, 2190,  754,  782, 4495, 3300,
          749, 3173, 4638, 2682, 3791,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}

In [94]:
res = model(**a).logits


In [72]:
print(res.shape)

torch.Size([1, 30, 21128])


In [95]:
q = torch.softmax(res, dim=2)
print(q.shape)

#c = torch.argmax(q, 2)
p = torch.topk(q, 10)[0][0][7]
c = torch.topk(q, 10)[-1][0][7]

print(p)
print(c)

for o, i in enumerate(c):
    print(p[o], tokenizer.decode(i.numpy().tolist()))

torch.Size([1, 14, 21128])
tensor([0.1687, 0.1631, 0.0940, 0.0820, 0.0777, 0.0298, 0.0244, 0.0226, 0.0196,
        0.0188], grad_fn=<SelectBackward0>)
tensor([5489, 7824, 4638, 7883, 7890, 4343, 6028, 7430, 7797, 5831])
tensor(0.1687, grad_fn=<SelectBackward0>) 肉
tensor(0.1631, grad_fn=<SelectBackward0>) 鱼
tensor(0.0940, grad_fn=<SelectBackward0>) 的
tensor(0.0820, grad_fn=<SelectBackward0>) 鸡
tensor(0.0777, grad_fn=<SelectBackward0>) 鸭
tensor(0.0298, grad_fn=<SelectBackward0>) 猪
tensor(0.0244, grad_fn=<SelectBackward0>) 蛋
tensor(0.0226, grad_fn=<SelectBackward0>) 雞
tensor(0.0196, grad_fn=<SelectBackward0>) 魚
tensor(0.0188, grad_fn=<SelectBackward0>) 菜


tensor(0.9958, grad_fn=<SelectBackward0>) 今
tensor(0.0042, grad_fn=<SelectBackward0>) 昨
tensor(5.4320e-06, grad_fn=<SelectBackward0>) 这
tensor(2.3714e-06, grad_fn=<SelectBackward0>) 刚
tensor(2.2109e-06, grad_fn=<SelectBackward0>) 现
tensor(1.5519e-06, grad_fn=<SelectBackward0>) 明
tensor(7.2464e-07, grad_fn=<SelectBackward0>) 去
tensor(5.9387e-07, grad_fn=<SelectBackward0>) 這
tensor(3.3581e-07, grad_fn=<SelectBackward0>) 一
tensor(2.8186e-07, grad_fn=<SelectBackward0>) 我


In [61]:
import torch
from chinesebert import ChineseBertForMaskedLM, ChineseBertTokenizerFast, ChineseBertConfig

pretrained_model_name = "junnyu/ChineseBERT-base"

tokenizer = ChineseBertTokenizerFast.from_pretrained(pretrained_model_name)
chinese_bert = ChineseBertForMaskedLM.from_pretrained(pretrained_model_name)

text = "北京是[MASK]国的首都。"
inputs = tokenizer(text, return_tensors="pt")
print(inputs)
maskpos = 4

with torch.no_grad():
    o = chinese_bert(**inputs)
    value, index = o.logits.softmax(-1)[0, maskpos].topk(10)

pred_tokens = tokenizer.convert_ids_to_tokens(index.tolist())
pred_values = value.tolist()

outputs = []
for t, p in zip(pred_tokens, pred_values):
    outputs.append(f"{t}|{round(p,4)}")
print(outputs)

# base  ['中|0.711', '我|0.2488', '祖|0.016', '法|0.0057', '美|0.0048', '全|0.0042', '韩|0.0015', '英|0.0011', '两|0.0008', '王|0.0006']
# large ['中|0.8341', '我|0.1479', '祖|0.0157', '全|0.0007', '国|0.0005', '帝|0.0001', '该|0.0001', '法|0.0001', '一|0.0001', '咱|0.0001']


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'ChineseBertTokenizerFast'.


KeyboardInterrupt: 

In [5]:
from transformers import DataCollatorWithPadding

from chinesebert import ChineseBertTokenizerFast

tokenizer = ChineseBertTokenizerFast.from_pretrained("junnyu/ChineseBERT-base")
collate_fn = DataCollatorWithPadding(tokenizer)
textlist = ["弗洛伊德的悲剧凸显了在美国和世界范围", "紧迫性和重要性，国际社会必须立", "那些存在严重种族主义、种族歧视", "中方对巴基斯坦开普省发"]
batch_list = [tokenizer(t) for t in textlist]
for i in textlist:
  print(tokenizer(i))
  break
batch = collate_fn(batch_list)
#print(batch.to("cuda:0"))

for i, e in enumerate(batch):

  print(e["pinyin_ids"] == tokenizer(textlist[i])["pinyin_ids"])



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'ChineseBertTokenizerFast'.


{'input_ids': [101, 2472, 3821, 823, 2548, 4638, 2650, 1196, 1137, 3227, 749, 1762, 5401, 1744, 1469, 686, 4518, 5745, 1741, 102], 'pinyin_ids': [0, 0, 0, 0, 0, 0, 0, 0, 11, 26, 2, 0, 0, 0, 0, 0, 17, 26, 20, 4, 0, 0, 0, 0, 30, 14, 1, 0, 0, 0, 0, 0, 9, 10, 2, 0, 0, 0, 0, 0, 9, 10, 5, 0, 0, 0, 0, 0, 7, 10, 14, 1, 0, 0, 0, 0, 15, 26, 4, 0, 0, 0, 0, 0, 25, 26, 1, 0, 0, 0, 0, 0, 29, 14, 6, 19, 3, 0, 0, 0, 17, 10, 5, 0, 0, 0, 0, 0, 31, 6, 14, 4, 0, 0, 0, 0, 18, 10, 14, 3, 0, 0, 0, 0, 12, 26, 20, 2, 0, 0, 0, 0, 13, 10, 2, 0, 0, 0, 0, 0, 24, 13, 14, 4, 0, 0, 0, 0, 15, 14, 10, 4, 0, 0, 0, 0, 11, 6, 19, 4, 0, 0, 0, 0, 28, 10, 14, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


TypeError: string indices must be integers

In [ ]:
"""[ 0,  0,  0,  0,  0,  0,  0,  0, 11, 26,  2,  0,  0,  0,  0,  0, 17, 26,
         20,  4,  0,  0,  0,  0, 30, 14,  1,  0,  0,  0,  0,  0,  9, 10,  2,  0,
          0,  0,  0,  0,  9, 10,  5,  0,  0,  0,  0,  0,  7, 10, 14,  1,  0,  0,
          0,  0, 15, 26,  4,  0,  0,  0,  0,  0, 25, 26,  1,  0,  0,  0,  0,  0,
         29, 14,  6, 19,  3,  0,  0,  0, 17, 10,  5,  0,  0,  0,  0,  0, 31,  6,
         14,  4,  0,  0,  0,  0, 18, 10, 14,  3,  0,  0,  0,  0, 12, 26, 20,  2,
          0,  0,  0,  0, 13, 10,  2,  0,  0,  0,  0,  0, 24, 13, 14,  4,  0,  0,
          0,  0, 15, 14, 10,  4,  0,  0,  0,  0, 11,  6, 19,  4,  0,  0,  0,  0,
         28, 10, 14,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],"""

In [5]:
import torch
import torch.nn as nn

In [9]:
input1 = torch.randn(100, 128)
input2 = torch.randn(100, 128)
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
output = cos(input1, input2)

In [13]:
a = cos(torch.tensor([0.0]), torch.tensor([0.0]))

In [17]:
torch.mean(torch.tensor([0.0]))

tensor(0.)

In [1]:
from transformers import AutoTokenizer

tokenizer_model_name_path="hfl/chinese-roberta-wwm-ext"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name_path)


In [18]:
tokenizer.decode([103])

'[MASK]'

In [19]:
import pickle

with open("tmp.pkl", "rb") as f:
    score = pickle.load(f)


In [31]:
print(score.shape)

torch.Size([52, 21128])


In [37]:
topk3 = score.topk(3, dim=1)[-1]

In [38]:
topk3

tensor([[2769,  511,  872],
        [ 872, 2644, 2769],
        [1962,  812, 2769],
        [ 106,  131,  117],
        [2769, 6443, 2218],
        [3221, 1373, 4263],
        [2476, 2769, 2484],
        [4263,  136, 2695],
        [3152, 2769,  872],
        [ 511, 8013,  117],
        [2769,  511,  872],
        [ 511, 8013,  106],
        [ 511,  106,  136],
        [ 511,  106, 8013],
        [ 511,  106, 8013],
        [ 511, 8013,  106],
        [ 511,  106, 8013],
        [ 511, 8013,  106],
        [ 511, 8013,  106],
        [ 511, 8013,  106],
        [ 106, 2769,  511],
        [ 511, 8013,  106],
        [ 511, 8013,  106],
        [ 511, 8013,  106],
        [ 511, 8013,  106],
        [ 511, 8013,  106],
        [2769,  872, 6468],
        [ 511, 8013,  106],
        [2769,  872, 6468],
        [ 872, 2769, 6468],
        [2769,  872, 6468],
        [ 511, 8013,  106],
        [2769,  872, 6468],
        [ 136,  511,  106],
        [ 511,  106, 8013],
        [ 511, 8013,

In [4]:
text = """Dataset Method Detection Level Correction Level
Pre Rec F1 Pre Rec F1
SIGHAN14
Hybrid(Wang et al., 2018) 51.9 66.2 58.2 - - 56.1
FASpell(Hong et al., 2019) 61.0 53.5 57.0 59.4 - -
SpellGCN(Cheng et al., 2020) 65.1 69.5 67.2 63.1 67.2 65.3
BERT(Xu et al., 2021) - - - 63.7 67.5 65.6
RoBERTa - - - 62.6 67.3 64.8
ChineseBERT - - - 63.7 68.3 65.9
PLOME(Liu et al., 2021) - - - - - -
REALISE(Xu et al., 2021) 67.8 71.5 69.6 66.3 70.0 68.1
mask-BERT - - - 63.0 63.0 63.0
mask-RoBERTa - - - 64.5 64.5 64.5
mask-ChineseBERT - - - 63.1 63.1 63.1
mask-PLOME - - - 63.1 63.1 63.1
mask-REALISE - - - 68.5 68.5 62.6
SIGHAN15
Hybrid(Wang et al., 2018) 56.6 69.4 62.3 - - 57.1
FASpell(Hong et al., 2019) 67.6 60.0 63.5 66.6 59.1 62.6
SpellGCN(Cheng et al., 2020) 74.8 80.7 77.7 72.1 77.7 75.9
BERT(Xu et al., 2021) 74.2 78.0 76.1 71.6 75.3 73.4
RoBERTa - - - 71.2 75.9 73.5
ChineseBERT - - - 72.1 76.7 74.3
PLOME(Liu et al., 2021) 77.4 81.5 79.4 75.3 79.3 77.2
REALISE(Xu et al., 2021) 77.3 81.3 79.3 75.9 79.9 77.8
mask-BERT - - - 67.5 67.5 63.0
mask-RoBERTa - - - 66.2 66.2 66.2
mask-ChineseBERT - - - 70.4 70.4 70.4
mask-PLOME - - - 63.1 63.1 63.1
mask-REALISE - - - 68.5 68.5 68.5"""

In [5]:
new_text = text.replace(" ", ",")

In [8]:
print(new_text)

Dataset,Method,Detection,Level,Correction,Level
Pre,Rec,F1,Pre,Rec,F1
SIGHAN14
Hybrid(Wang,et,al.,,2018),51.9,66.2,58.2,-,-,56.1
FASpell(Hong,et,al.,,2019),61.0,53.5,57.0,59.4,-,-
SpellGCN(Cheng,et,al.,,2020),65.1,69.5,67.2,63.1,67.2,65.3
BERT(Xu,et,al.,,2021),-,-,-,63.7,67.5,65.6
RoBERTa,-,-,-,62.6,67.3,64.8
ChineseBERT,-,-,-,63.7,68.3,65.9
PLOME(Liu,et,al.,,2021),-,-,-,-,-,-
REALISE(Xu,et,al.,,2021),67.8,71.5,69.6,66.3,70.0,68.1
mask-BERT,-,-,-,63.0,63.0,63.0
mask-RoBERTa,-,-,-,64.5,64.5,64.5
mask-ChineseBERT,-,-,-,63.1,63.1,63.1
mask-PLOME,-,-,-,63.1,63.1,63.1
mask-REALISE,-,-,-,68.5,68.5,62.6
SIGHAN15
Hybrid(Wang,et,al.,,2018),56.6,69.4,62.3,-,-,57.1
FASpell(Hong,et,al.,,2019),67.6,60.0,63.5,66.6,59.1,62.6
SpellGCN(Cheng,et,al.,,2020),74.8,80.7,77.7,72.1,77.7,75.9
BERT(Xu,et,al.,,2021),74.2,78.0,76.1,71.6,75.3,73.4
RoBERTa,-,-,-,71.2,75.9,73.5
ChineseBERT,-,-,-,72.1,76.7,74.3
PLOME(Liu,et,al.,,2021),77.4,81.5,79.4,75.3,79.3,77.2
REALISE(Xu,et,al.,,2021),77.3,81.3,79.3,75.9,79.9,77.